In [1]:
from dataclasses import dataclass
import numpy as np
import torch

from reflex_model import GPT

In [2]:
# @dataclass
# class GPTConfig:
#     batch_size: int = 8
#     block_size: int = 2048
#     vocab_size: int = 50257
#     n_layer: int = 4
#     n_head: int = 4
#     n_embd: int = 1536
#     dropout: float = 0.1
#     bias: bool=True
#     pretrained_model_path: str="/home/user/models/rugpt"
#     local_files_only: bool=True

@dataclass
class GPTConfig:
    batch_size: int = 8
    block_size: int = 1024
    vocab_size: int = 10000
    n_layer: int = 4
    n_head: int = 4
    n_embd: int = 1024
    dropout: float = 0.1
    bias: bool=True
    pretrained_model_path: str="/home/user/models/rugpt"
    local_files_only: bool=True

In [3]:
config = GPTConfig()
model = GPT(config)

Partially copied: transformer.wte.weight -> transformer.wte.weight (shape torch.Size([50257, 1536]) -> torch.Size([10000, 1024]))
Partially copied: lm_head.weight -> lm_head.weight (shape torch.Size([50257, 1536]) -> torch.Size([10000, 1024]))
Partially copied: transformer.wpe.weight -> transformer.wpe.weight (shape torch.Size([2048, 1536]) -> torch.Size([1024, 1024]))
Partially copied: transformer.ln_f.weight -> transformer.ln_f.weight (shape torch.Size([1536]) -> torch.Size([1024]))
Partially copied: transformer.ln_f.bias -> transformer.ln_f.bias (shape torch.Size([1536]) -> torch.Size([1024]))
Partially copied: transformer.h.0.ln_1.weight -> transformer.h.0.ln_1.weight (shape torch.Size([1536]) -> torch.Size([1024]))
Partially copied: transformer.h.0.ln_2.weight -> transformer.h.0.ln_2.weight (shape torch.Size([1536]) -> torch.Size([1024]))
Partially copied: transformer.h.0.attn.c_attn.weight -> transformer.h.0.attn.c_attn.weight (shape torch.Size([4608, 1536]) -> torch.Size([3072, 

In [4]:
params_total = model.get_num_params()
params_bytes = params_total*4
params_and_buffers_bytes = params_bytes + 2*params_bytes
print(f"est checkpoint size: {params_and_buffers_bytes/1e9:.2f} GB")
print(f"{params_total/1e6:.1f} millions of params")

est checkpoint size: 2.34 GB
194.9 millions of params
